In [1]:
%run "/code/source/notebooks/notebook_settings.py"
import source.library.openai as oai
from source.service.datasets import DATA
# This is needed because openai.text_completion calls asynchronous functions but 
# Jupyter is already running its own event loop.
import nest_asyncio
nest_asyncio.apply()

with open('/.openai_api_key', 'r') as handle:
    oai.API_KEY = handle.read().strip()

reddit = DATA.reddit.load()

2023-04-09 21:18:49 - INFO     | Loading data `reddit` from `data/processed/reddit.pkl`


In [2]:
reddit.head()

,id,subreddit,title,post,text
0,6y0l40,Volvo,2012 c30 spoiler holding water,Lately my car has been a magnet for for dings and other mishaps while parked in my reserved spot and have been parking elsewhere which is unfortunately under a tree. I've noticed after a night of ...,2012 c30 spoiler holding water - Lately my car has been a magnet for for dings and other mishaps while parked in my reserved spot and have been parking elsewhere which is unfortunately under a tre...
1,7w66o5,Volvo,"About to purchase 2014 V60, anything I should be aware of?","My 2004 V50 is slowly but surely giving up, it feels like I replaced half of the replaceable parts and got the other half repaired within the last year and I think it's time to let him go, especia...","About to purchase 2014 V60, anything I should be aware of? - My 2004 V50 is slowly but surely giving up, it feels like I replaced half of the replaceable parts and got the other half repaired with..."
2,6ujf4y,Volvo,Experiencing the new XC60,"I test drove the XC60 T8 yesterday and Oh. My. God is it quick. I was obviously aware of the performance numbers (bhp, 0-60 times etc), but it's a completely different experience trying it out for...","Experiencing the new XC60 - I test drove the XC60 T8 yesterday and Oh. My. God is it quick. I was obviously aware of the performance numbers (bhp, 0-60 times etc), but it's a completely different ..."
3,70pr0e,Volvo,Pilot Assist 2 upgrade 2016 xc90,I am looking to pick up a used 2016 and am really interested in the Pilot Assist 2 retro fit upgrade that can be added.<lb><lb>What do i need to look for to make sure the 2016 model i get can be r...,Pilot Assist 2 upgrade 2016 xc90 - I am looking to pick up a used 2016 and am really interested in the Pilot Assist 2 retro fit upgrade that can be added.<lb><lb>What do i need to look for to make...
4,4uqr9q,Volvo,2013 XC60 Negotiation with Volvo dealership,I am in the middle of a negotiation on the price of a 2013 XC60. We are off by 1k. I am debating about throwing out a request that he upgrades the software in my 2006 2.5 xc90 AWD with the polesta...,2013 XC60 Negotiation with Volvo dealership - I am in the middle of a negotiation on the price of a 2013 XC60. We are off by 1k. I am debating about throwing out a request that he upgrades the sof...


In [3]:
reddit.iloc[0]['text']

"2012 c30 spoiler holding water - Lately my car has been a magnet for for dings and other mishaps while parked in my reserved spot and have been parking elsewhere which is unfortunately under a tree. I've noticed after a night of rain when I leave the next day water runs down the back window when I accelerate. Opening the back glass before driving usually means water runs down the sides but I cannot seem to find where it's pooling up. <lb><lb>Anyone know if there's drains in the back hatch that might be clogged? Or is this normal?"

In [4]:
# cost to send all reddit posts e.g. as prompts... non-sensical but just getting an idea
print(f"{oai.cost(reddit.iloc[0]['text'], oai.InstructModels.BABBAGE) * len(reddit)}")

0.0585


In [5]:
def generate_prompt(post: str) -> str:
    template = f"""
    Summarize this reddit post:

    ```
    {post}
    ```
    """
    return template

posts = reddit['text'].iloc[0:10]
prompts = [generate_prompt(x) for x in posts]
responses = oai.text_completion(model=oai.InstructModels.BABBAGE, prompts=prompts, max_tokens=200)
responses

OpenAIResponses(responses=[OpenAIResponse(response_status=200, response_reason='OK', result=OpenAIInstructResult(result={'id': 'cmpl-73Wiw5sTkBVTv9GDgEH6lCEETk6Qs', 'object': 'text_completion', 'created': 1681075130, 'model': 'text-babbage-001', 'choices': [{'text': ' A c30 spoiler holding water is a common problem on a car parked in a reserved spot. The water can run down the back window if you open the back glass before driving. Sometimes the water pooled up in the back hatch and it was difficult to find where it was coming from.', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 149, 'completion_tokens': 57, 'total_tokens': 206}})), OpenAIResponse(response_status=200, response_reason='OK', result=OpenAIInstructResult(result={'id': 'cmpl-73WiwQi1KzKz6Lx8WuetuKVrHtAFj', 'object': 'text_completion', 'created': 1681075130, 'model': 'text-babbage-001', 'choices': [{'text': ' A V60 2014 model is being tested for a potential sale tomorrow. The owner is no

In [6]:
assert not responses.any_errors
assert not responses.any_missing_data

In [7]:
print(f"${responses.total_cost:.4f}")

$0.0014


In [8]:
print(responses[0].result.usage_prompt_tokens)
print(responses[0].result.usage_completion_tokens)


149
57


In [9]:
for response, post in zip(responses, posts):
    print(f"\n\n---------\nTotal Cost: ${response.result.cost_total:.5f}")
    print(
        f"Total Tokens: {response.result.usage_total_tokens}; "
        f"Prompt Tokens: {response.result.usage_prompt_tokens}; "
        f"Completion Tokens: {response.result.usage_completion_tokens}"
    )
    print(f"Post: `{post}`")
    print(f"Summary: `{response.result.reply}`")



---------
Total Cost: $0.00010
Total Tokens: 206; Prompt Tokens: 149; Completion Tokens: 57
Post: `2012 c30 spoiler holding water - Lately my car has been a magnet for for dings and other mishaps while parked in my reserved spot and have been parking elsewhere which is unfortunately under a tree. I've noticed after a night of rain when I leave the next day water runs down the back window when I accelerate. Opening the back glass before driving usually means water runs down the sides but I cannot seem to find where it's pooling up. <lb><lb>Anyone know if there's drains in the back hatch that might be clogged? Or is this normal?`
Summary: `A c30 spoiler holding water is a common problem on a car parked in a reserved spot. The water can run down the back window if you open the back glass before driving. Sometimes the water pooled up in the back hatch and it was difficult to find where it was coming from.`


---------
Total Cost: $0.00020
Total Tokens: 394; Prompt Tokens: 306; Completion

---